# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 10:55AM,256699,19,9123602,"Snap Medical Industries, LLC",Released
1,Feb 6 2023 10:55AM,256699,19,9123611,"Snap Medical Industries, LLC",Released
2,Feb 6 2023 10:54AM,256698,19,MSP218192,"Methapharm, Inc.",Released
3,Feb 6 2023 10:50AM,256696,12,7559000,"NBTY Global, Inc.",Released
4,Feb 6 2023 10:40AM,256695,15,9067722-BO,"Brookfield Pharmaceuticals, LLC",Released
5,Feb 6 2023 10:40AM,256695,15,9067727-BO,"Brookfield Pharmaceuticals, LLC",Released
6,Feb 6 2023 10:40AM,256695,15,9067732-BO,"Brookfield Pharmaceuticals, LLC",Released
7,Feb 6 2023 10:40AM,256695,15,9067737-BO,"Brookfield Pharmaceuticals, LLC",Released
8,Feb 6 2023 10:40AM,256695,15,9066628-BO,"Brookfield Pharmaceuticals, LLC",Released
9,Feb 6 2023 10:40AM,256695,15,9067714-BO,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,256694,Released,1
26,256695,Released,34
27,256696,Released,1
28,256698,Released,1
29,256699,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256694,NaN,1.0
256695,NaN,34.0
256696,NaN,1.0
256698,NaN,1.0
256699,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256651,1.0,0.0
256656,1.0,2.0
256662,0.0,1.0
256668,7.0,0.0
256670,0.0,35.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256651,1,0
256656,1,2
256662,0,1
256668,7,0
256670,0,35


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256651,1,0
1,256656,1,2
2,256662,0,1
3,256668,7,0
4,256670,0,35


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256651,1,
1,256656,1,2
2,256662,,1
3,256668,7,
4,256670,,35


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 10:55AM,256699,19,"Snap Medical Industries, LLC"
2,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc."
3,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc."
4,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC"
38,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions
39,Feb 6 2023 10:22AM,256686,16,Sartorius Stedim Filters Inc.
40,Feb 6 2023 10:15AM,256692,10,Methapharm-G
41,Feb 6 2023 10:14AM,256691,10,Bio-PRF
48,Feb 6 2023 10:13AM,256690,10,Eywa Pharma Inc.
50,Feb 6 2023 10:12AM,256689,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 6 2023 10:55AM,256699,19,"Snap Medical Industries, LLC",,2
1,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",,1
2,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",,1
3,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",,34
4,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions,,1
5,Feb 6 2023 10:22AM,256686,16,Sartorius Stedim Filters Inc.,,1
6,Feb 6 2023 10:15AM,256692,10,Methapharm-G,,1
7,Feb 6 2023 10:14AM,256691,10,Bio-PRF,,7
8,Feb 6 2023 10:13AM,256690,10,Eywa Pharma Inc.,,2
9,Feb 6 2023 10:12AM,256689,10,"Methapharm, Inc.",,26


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:55AM,256699,19,"Snap Medical Industries, LLC",2,
1,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1,
2,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",1,
3,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",34,
4,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions,1,
5,Feb 6 2023 10:22AM,256686,16,Sartorius Stedim Filters Inc.,1,
6,Feb 6 2023 10:15AM,256692,10,Methapharm-G,1,
7,Feb 6 2023 10:14AM,256691,10,Bio-PRF,7,
8,Feb 6 2023 10:13AM,256690,10,Eywa Pharma Inc.,2,
9,Feb 6 2023 10:12AM,256689,10,"Methapharm, Inc.",26,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:55AM,256699,19,"Snap Medical Industries, LLC",2,
1,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1,
2,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",1,
3,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",34,
4,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:55AM,256699,19,"Snap Medical Industries, LLC",2.0,NaN
1,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1.0,NaN
2,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",1.0,NaN
3,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",34.0,NaN
4,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 10:55AM,256699,19,"Snap Medical Industries, LLC",2.0,0.0
1,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1.0,0.0
2,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",1.0,0.0
3,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",34.0,0.0
4,Feb 6 2023 10:28AM,256694,16,Sartorius Bioprocess Solutions,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3080176,197.0,12.0
12,256696,1.0,0.0
15,256695,34.0,0.0
16,1026723,4.0,4.0
19,1026729,7.0,1.0
21,770052,3.0,0.0
22,256651,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3080176,197.0,12.0
1,12,256696,1.0,0.0
2,15,256695,34.0,0.0
3,16,1026723,4.0,4.0
4,19,1026729,7.0,1.0
5,21,770052,3.0,0.0
6,22,256651,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,197.0,12.0
1,12,1.0,0.0
2,15,34.0,0.0
3,16,4.0,4.0
4,19,7.0,1.0
5,21,3.0,0.0
6,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,197.0
1,12,Released,1.0
2,15,Released,34.0
3,16,Released,4.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Executing,12.0,0.0,0.0,4.0,1.0,0.0,1.0
Released,197.0,1.0,34.0,4.0,7.0,3.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Executing,12.0,0.0,0.0,4.0,1.0,0.0,1.0
1,Released,197.0,1.0,34.0,4.0,7.0,3.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Executing,12.0,0.0,0.0,4.0,1.0,0.0,1.0
1,Released,197.0,1.0,34.0,4.0,7.0,3.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()